##                                      MACHINE LEARNING
###                                                           Project 1

###                                          Evangelia Santorinaiou 3130182

#### Libraries

In [116]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image


### Load Data

In [117]:
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

In [118]:
def get_cifar_data(name="train"):
    x = None
    y = None
    l = None

    f = open('cifar-10-batches-py/batches.meta', 'rb')
    datadict = pickle.load(f, encoding='latin1')
    f.close()
    l = datadict['label_names']

    if name is "train":
        for i in range(5):                         #5 train data
            f = open('cifar-10-batches-py/data_batch_' + str(i + 1), 'rb')
            datadict = pickle.load(f, encoding='latin1')
            f.close()

            _X = datadict["data"]
            _Y = datadict['labels']

            _X = np.array(_X, dtype=float) / 255.0 #create a vector for each example
            _X = _X.reshape([-1, 3, 32, 32])       #32x32 are the dimensions of each image, 3 pixels for the RGB
                                                   #and -1 for keeping the first dimension, here the examples.
            _X = _X.transpose([0, 2, 3, 1])
            _X = _X.reshape(-1, 32 * 32 * 3)       #reshape again to a 2-dimension vector

            if x is None:
                x = _X
                y = _Y
            else:
                x = np.concatenate((x, _X), axis=0)
                y = np.concatenate((y, _Y), axis=0)

    elif name is "test":
        f = open('./data_set/' + folder_name + '/test_batch', 'r')
        datadict = pickle.load(f, encoding='latin1')
        f.close()

        x = datadict["data"]
        y = np.array(datadict['labels'])

        x = np.array(x, dtype=float) / 255.0
        x = x.reshape([-1, 3, 32, 32])
        x = x.transpose([0, 2, 3, 1])
        x = x.reshape(-1, 32*32*3)

    def dense_to_one_hot(labels_dense, num_classes=10):
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1

        return labels_one_hot

    return x, dense_to_one_hot(y), l


In [119]:
def get_mnist_data(name="train"):
    
    train_files = ['mnistdata/train%d.txt' % (i,) for i in range(9)]
    test_files = ['mnistdata/test%d.txt' % (i,) for i in range(9)]
    tmp = []
    
    for i in train_files:
        with open(i, 'r') as fp:
            tmp += fp.readlines()
    
    
    train_data = np.array([[j for j in i.split(" ")] for i in tmp], dtype='int') / 255
    print "Train data array size: ", train_data.shape
    tmp = []
    for i in test_files:
        with open(i, 'r') as fp:
            tmp += fp.readlines()
            
            
    test_data = np.array([[j for j in i.split(" ")] for i in tmp], dtype='int') / 255
    print "Test data array size: ", test_data.shape
    tmp = []
    for i, _file in enumerate(train_files):
        with open(_file, 'r') as fp:
            for line in fp:
                tmp.append([1 if j == i else 0 for j in range(0, 10)])
    
    train_truth = np.array(tmp, dtype='int')
    del tmp[:]
    for i, _file in enumerate(test_files):
        with open(_file, 'r') as fp:
            for _ in fp:
                tmp.append([1 if j == i else 0 for j in range(0, 10)])
    test_truth = np.array(tmp, dtype='int')
    print "Train truth array size: ", train_truth.shape
    print "Test truth array size: ", test_truth.shape
    
    return train_data, test_data, train_truth, test_truth
    
    

#### Neural Network Classifier implementation

##### Basic Functions:
* **neural_network_model** splits the data into baches and does the training process using gradient ascent to minimize the cost function via back propagation. 
* **test_neural_network_model**  compares the true values with the predicted ones it got from the trained model and 
compute the average error
* **predict** does the testing by predicting the classes of unknown data 
* **gradcheck** tests the analytical gradients computation by comparing them to a sample of numerical gradient computations


#####    Helper Functions:
   Implemented functions in a numerically stable manner
* **softmax** 
* **softplus** 
* **tanh**
* **cos**

Derivatives of the above functions
* **d_softplus**
* **d_tanh**
* **d_cos**<br><br>

* **cost_function** 

$$E(w) = \sum_{n=1}^{N}\sum_{k=1}^{K}t_{nk}y_{nk} - \frac{\lambda}{2}(\lVert w^{(1)} \rVert^2 + \lVert w^{(2)} \rVert^2)$$<br><br>
- **forward_propagation**  executes the *forward propagation* in the network: <br><br>
    Input propagating to the hidden layer: $ \ \ \ \ \ $ $A_1 = X(W^{(1)})^T$ <br>
    Activation in the hidden layer: $\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ $ $Z = H(A_1) + ones$ <br>
    Propagation from hidden layer to output: $\  $ $A_2 = Z(W^{(2)})^T$ <br><br>
    It is called twice each time, once for the 1st layer until the new representation emerges from the 1st activation, and then for the second layer where we use softmax to calculate the output<br><br>
    
- **grad_dW1**  implementation of the gradient of $E(w)$ against $W^{(1)}$ <br><br>
$$\frac{\partial}{\partial W^{1}} E(W) = ((T-Y)W^{(2)}\circ H)^T)X - \lambda W^{(1)} $$ <br><br>
- **grad_dW2**  implementation of the gradient of $E(w)$ against $W^{(2)}$ <br><br>
$$\frac{\partial}{\partial W^{2}} E(W) = (T-Y)^TX - \lambda W^{(2)}$$<br><br>


* **get_batches**  creates batches from shuffled data


### Activation functions

In [120]:
def softmax(y):
    max_of_rows = np.max(y, 1)
    m = np.array([max_of_rows, ] * y.shape[1]).T
    y = y - m
    y = np.exp(y)
    return y / ((np.array([np.sum(y, 1), ] * y.shape[1])).T) * sd


In [121]:
def softplus(a):
    max_of_rows = np.max(a, 0)
    m = np.array([max_of_rows, ] * a.shape[0])
    return m + np.log(np.exp(-m) + np.exp(a - m)) 
    
    
def tanh(a):
    max_of_rows = np.max(a, 1)
    m = np.array([max_of_rows, ] * a.shape[1]).T
    return (np.exp(a - m) - np.exp(-a - m)) / (np.exp(a - m) + np.exp(-a - m))
    
    
def cos(a):
    return np.cos(a)

### Activation Derivatives

In [122]:
def d_softplus(a):
    return 1 / (1 + np.exp(-a))
    
def d_tanh(a):
    return (1 - np.square(tanh(a)))
    
def d_cos(a):
    return - np.sin(a)
    

## Neural Network Model

In [123]:
def weight_initialization(M, D, K):
    W1 = np.random.randn(M, D + 1) / np.sqrt(D + 1)  
    W2 = np.random.randn(K, M + 1) / np.sqrt(M + 1) 
    return W1, W2

In [124]:
def cost_function(W1, W2, A2, T, lamda):
    S = softmax(A2)
    M = np.max(S, axis=1)
    
    return np.sum(T * A2) - np.sum(M) - np.sum(np.log(np.sum(np.exp(A2 - np.array([M, ] * A2.shape[1]).T), 1))) - \
           (0.5 * lamda) * (np.sum(np.square(W1)) +  np.sum(np.square(W2)))

In [125]:
def grad_dW1(X, W1, W2, Y, T, lamda, A1, activation_function):
   
    if activation_function == "softplus":
        H = d_softplus(A1)
    elif activation_function == "tanh":
        H = d_tanh(A1)
    else:  # cos
        H = d_cos(A1)
    return np.dot(np.multiply(np.dot((T - Y), W2[:,1:]), H).T, X) - lamda * W1

In [126]:
def grad_dW2(Z, W2, Y, T, lamda): 
    return np.dot((T - Y).T, Z) - lamda * W2

In [127]:
def forward_propagation(X, W, activation_function):
    ones = np.ones((X.shape[0], 1))
    X = np.append(ones, X, axis=1)
    A = np.dot(X, W.T)

    if activation_function   == "softplus":
        Z = softplus(A)
    elif activation_function == "tanh":
        Z = tanh(A)
    elif activation_function == "cos":
        Z = cos(A)
    elif activation_function == "softmax":
        Z = softmax(A)
    else:
        print("'%s' is not supported activation function" % activation_function)
        exit()

    return X, A, Z

In [128]:
def back_propagation(X, W1, W2, Y, T, Z, lamda, A1, A2, eta, activation_function):  
    # update parameters
    W1 += eta * grad_dW1(X, W1, W2, Y, T, lamda, A1, activation_function)  
    W2 += eta * grad_dW2(Z, W2, Y, T, lamda)
    
    return W1, W2

In [129]:
def get_batches(X, T, b_size):             # b_size: mini-batch size: 16, 32, 64, 128, 256, ...
    #Shuffle X and Y
    N = X.shape[0]
    perm = list(np.random.permutation(N))  #indexes of random shuffle
    X_shuffled = X[perm, :]                #shuffled data
    T_shuffled = T[perm, :]                #same perm because we want the respective labels
    
    #Partition of X_shuffled and Y_shuffled
    batches = []
    com_batches =  N // b_size              # complete batches
    for i in range(com_batches):
        X_batch = X_shuffled[i*b_size:(i+1)*b_size, :]  
        T_batch = T_shuffled[i*b_size:(i+1)*b_size, :]
        batches.append((X_batch, T_batch))  

    #for the last batch in case we have one
    if N % b_size != 0:
        X_batch = X_shuffled[com_batches*b_size:, :]  
        T_batch = T_shuffled[com_batches*b_size:, :]
        batches.append((X_batch, T_batch))  

    return batches

In [143]:
def neural_network_model(X, T, activation_function, lamda, M, number_of_iterations, b_size, learning_rate): 
    
    W1, W2 = weight_initialization(M, X.shape[1], T.shape[1])
    Ew_old = float('inf')
    eta = learning_rate / b_size
    
    for i in range(number_of_iterations):
        batches = get_batches(X, T, b_size)
        for batch in batches:
            (X_batch, T_batch) = batch
            # 1st step forward
            X_batch, A1_batch, Z_batch = forward_propagation(X_batch, W1, activation_function)
            # 2nd step forward
            Z_batch, A2_batch, Y_batch = forward_propagation(Z_batch, W2, 'softmax')
            Ew     = cost_function(W1, W2, A2_batch, T_batch, lamda)
            W1, W2 = back_propagation(X_batch, W1, W2, Y_batch, T_batch, Z_batch, lamda, A1_batch, A2_batch, eta, activation_function)

        if np.abs(Ew - Ew_old) < 1e-6:
                break
        Ew_old = Ew
        
        print("Iteration: %d   Cost: %f" % (i, Ew))

    return W1, W2

In [131]:
def predict(X, W1, W2, activation_function):
    _, _, Z = forward_propagation(X, W1, activation_function)
    _, _, Y = forward_propagation(Z, W2, 'softmax')
    return np.argmax(Y, axis=1)

In [132]:
def test_neural_network_model(X_test, T_test, W1, W2, activation_function):
    T_predict = predict(X_test, W1, W2, activation_function)
    T_true = np.argmax(T_test, axis=1)
    error = np.mean(T_predict != T_true)
    print("Error: %d%%" %(error*100))

In [133]:
def gradcheck(X, W1, W2, T, lamda, activation_function):
   
    epsilon  = 1e-6
    list_    = np.random.randint(X.shape[0], size=10)
    
    X_sample  = np.array(X[list_, :])
    T_sample  = np.array(T[list_, :])     
    
    X_sample, A1, Z_sample = forward_propagation(X_sample, W1, activation_function)
    Z_sample, A2, Y_sample = forward_propagation(Z_sample, W2, 'softmax')
    
    Ew = cost_function(W1, W2, A2, T_sample, lamda)
    
    grad_dW1_ = grad_dW1(X_sample, W1, W2, Y_sample, T_sample, lamda, A1, activation_function)
    numericalGrad_W1 = np.zeros(grad_dW1_.shape)
    
    grad_dW2_ = grad_dW2(Z_sample, W2, Y_sample, T_sample, lamda)
    numericalGrad_W2 = np.zeros(grad_dW2_.shape)
    
    # Compute all numerical gradient estimates and store them in
    # the matrix numericalGrad_W1
    for k in range(numericalGrad_W1.shape[0]):
        for d in range(numericalGrad_W1.shape[1]):
            W1_tmp1 = np.copy(W1)
            W1_tmp1[k, d] += epsilon
            e_plus  = cost_function(W1_tmp1, W2, A2, T_sample, lamda)
            
            W1_tmp2 = np.copy(W1)
            W1_tmp2[k, d] -= epsilon
            e_minus = cost_function(W1_tmp2, W2, A2, T_sample, lamda)
            
            numericalGrad_W1[k, d] = (e_plus - e_minus) / (2 * epsilon)
    
    
    # Compute all numerical gradient estimates and store them in
    # the matrix numericalGrad_W2
    for k in range(numericalGrad_W2.shape[0]):
        for d in range(numericalGrad_W2.shape[1]):
            W2_tmp1 = np.copy(W2)
            W2_tmp1[k, d] += epsilon
            e_plus = cost_function(W1, W2_tmp1, A2, T_sample, lamda)
            
            W2_tmp2 = np.copy(W2)
            W2_tmp2[k, d] -= epsilon
            e_minus = cost_function(W1, W2_tmp2, A2, T_sample, lamda)
            
            numericalGrad_W2[k, d] = (e_plus - e_minus) / (2 * epsilon)
    
    # Absolute norm
    print(np.max(np.abs(grad_dW1_ - numericalGrad_W1)))
    print(np.max(np.abs(grad_dW2_ - numericalGrad_W2)))

### Testing the Model

In [134]:
X_train, X_test, T_train, T_test = get_mnist_data(name="train")
#X_train, X_test, T_train, T_test = get_cifar_data(name="train")

Train data array size:  (54051, 784)
Test data array size:  (8991, 784)
Train truth array size:  (54051, 10)
Test truth array size:  (8991, 10)


In [135]:
#Load part of the data in order to check it faster
N = X_train.shape[0]
choice = np.random.choice(N, size=60000)
X_ = X_train[choice,:]
T_ = T_train[choice,:]

In [136]:
activation_f = 'softplus'

### Simple Validation


In [137]:
def test_performance(T_val,Y_val):
    return np.sqrt(np.sum(np.square(Y_val - T_val))/np.linalg.norm(Y_val))

In [141]:
def simple_validation(X, T, M, activation_f):
    
    N = X.shape[0]
    a = int(N * 0.2)
    X_val = X[:a, :]
    T_val = T[:a, :]
    
    X_train = X[a:, :]
    T_train = T[a:, :]
    
    lamdas = [0.01, 0.002, 0.004 ,0.006, 0.008]    
    min_e =  float('inf')
    the_one = lamdas[0]
    
    for lamda in lamdas:
        W1, W2 = neural_network_model(X_train, T_train, activation_f, lamda,  M, number_of_iterations, b_size, learning_rate)
        _, _, Z = forward_propagation(X_val, W1, activation_f)
        _, _, Y_val = forward_propagation(Z, W2, 'softmax')
        e = test_performance(T_val, Y_val)
        if e < min_e:
            min_e = e
            the_one = lamda
        
    print "The one is ", the_one
   
    return the_one 

As we notice in the board below the error is quiet big. This is because the gradient check gives 
an important diferrence between the real gradient and the one we compute. So as we add neurals the error
is multiplied and we have bigger final error.

The two tables below have been computed for 100 iterations and 0.001 learning rate. The lamdas have been chosen
in simple validation seperately for each case.

**Errors

|        | 100 | 200 | 300|         
|--------|-----|-----|----|
|softplus|  7  |  8  | 8  |
|--------|-----|-----|----|
|cos     |  7  |  7  | 7  |
|--------|-----|-----|----|
|tanh    |  7  |  7  | 7  |

**Lamdas

|        | 100 | 200  | 300 |
|        |     |      |     |
|softplus|0.002|0.008 |0.008|
|        |     |      |     |
|cos     |0.002|0.008 |0.04 |
|        |     |      |     |
|tanh    |0.01 |0.06  |0,006|

In [145]:
M = [100, 200, 300]
function = ["softplus", "cos", "tanh"]
number_of_iterations = 100
b_size = 128
learning_rate = 0.001

for m in M:
    for f in function:
        print("Function %s M =%d" %(f, m))
        lamda = simple_validation(X_, T_, m, f)
        W1, W2 = neural_network_model(X_, T_, f, lamda,  m, number_of_iterations, b_size, learning_rate)
        test_neural_network_model(X_, T_, W1, W2, f)
        gradcheck(X_, W1, W2, T_, lamda, f)

Function softplus M =100
Iteration: 0   Cost: -274.963692
Iteration: 1   Cost: -252.981415
Iteration: 2   Cost: -239.456012
Iteration: 3   Cost: -225.727569
Iteration: 4   Cost: -203.649112
Iteration: 5   Cost: -201.037299
Iteration: 6   Cost: -169.912766
Iteration: 7   Cost: -161.903377
Iteration: 8   Cost: -159.995853
Iteration: 9   Cost: -141.146936
Iteration: 10   Cost: -127.408960
Iteration: 11   Cost: -133.870685
Iteration: 12   Cost: -109.970895
Iteration: 13   Cost: -101.526683
Iteration: 14   Cost: -109.270779
Iteration: 15   Cost: -98.736221
Iteration: 16   Cost: -91.061422
Iteration: 17   Cost: -82.943562
Iteration: 18   Cost: -84.077245
Iteration: 19   Cost: -74.214455
Iteration: 20   Cost: -72.747151
Iteration: 21   Cost: -75.791452
Iteration: 22   Cost: -59.457348
Iteration: 23   Cost: -75.749762
Iteration: 24   Cost: -64.049358
Iteration: 25   Cost: -66.140332
Iteration: 26   Cost: -64.541011
Iteration: 27   Cost: -45.730562
Iteration: 28   Cost: -66.891332
Iteration: 29

Iteration: 48   Cost: -48.399318
Iteration: 49   Cost: -54.989222
Iteration: 50   Cost: -50.885853
Iteration: 51   Cost: -38.300968
Iteration: 52   Cost: -58.983062
Iteration: 53   Cost: -55.871828
Iteration: 54   Cost: -48.883081
Iteration: 55   Cost: -38.517579
Iteration: 56   Cost: -35.463909
Iteration: 57   Cost: -31.961843
Iteration: 58   Cost: -32.039174
Iteration: 59   Cost: -48.660816
Iteration: 60   Cost: -34.631627
Iteration: 61   Cost: -52.698961
Iteration: 62   Cost: -52.323751
Iteration: 63   Cost: -43.472099
Iteration: 64   Cost: -65.684100
Iteration: 65   Cost: -47.305641
Iteration: 66   Cost: -32.469409
Iteration: 67   Cost: -46.155166
Iteration: 68   Cost: -44.364688
Iteration: 69   Cost: -47.460233
Iteration: 70   Cost: -39.141363
Iteration: 71   Cost: -39.704984
Iteration: 72   Cost: -39.396583
Iteration: 73   Cost: -50.969601
Iteration: 74   Cost: -43.518833
Iteration: 75   Cost: -66.813315
Iteration: 76   Cost: -28.005018
Iteration: 77   Cost: -37.193612
Iteration:

Iteration: 97   Cost: -49.979235
Iteration: 98   Cost: -39.145922
Iteration: 99   Cost: -33.206967
The one is  0.002
Iteration: 0   Cost: -200.130858
Iteration: 1   Cost: -187.866159
Iteration: 2   Cost: -168.126389
Iteration: 3   Cost: -157.707884
Iteration: 4   Cost: -141.845956
Iteration: 5   Cost: -127.264008
Iteration: 6   Cost: -115.021242
Iteration: 7   Cost: -102.612526
Iteration: 8   Cost: -92.386020
Iteration: 9   Cost: -98.661249
Iteration: 10   Cost: -77.064271
Iteration: 11   Cost: -74.159065
Iteration: 12   Cost: -68.883117
Iteration: 13   Cost: -62.134384
Iteration: 14   Cost: -67.179832
Iteration: 15   Cost: -59.107539
Iteration: 16   Cost: -62.794218
Iteration: 17   Cost: -54.924559
Iteration: 18   Cost: -55.248352
Iteration: 19   Cost: -57.915590
Iteration: 20   Cost: -54.820647
Iteration: 21   Cost: -56.333080
Iteration: 22   Cost: -43.142865
Iteration: 23   Cost: -44.709323
Iteration: 24   Cost: -42.336199
Iteration: 25   Cost: -42.335258
Iteration: 26   Cost: -24.7

Iteration: 43   Cost: -50.962085
Iteration: 44   Cost: -60.190261
Iteration: 45   Cost: -39.103656
Iteration: 46   Cost: -43.426242
Iteration: 47   Cost: -54.317817
Iteration: 48   Cost: -43.576470
Iteration: 49   Cost: -58.453655
Iteration: 50   Cost: -42.184735
Iteration: 51   Cost: -50.042450
Iteration: 52   Cost: -47.934929
Iteration: 53   Cost: -37.960228
Iteration: 54   Cost: -45.369695
Iteration: 55   Cost: -37.757498
Iteration: 56   Cost: -35.415559
Iteration: 57   Cost: -49.365586
Iteration: 58   Cost: -42.147303
Iteration: 59   Cost: -37.929057
Iteration: 60   Cost: -37.426138
Iteration: 61   Cost: -41.125516
Iteration: 62   Cost: -34.528146
Iteration: 63   Cost: -33.108030
Iteration: 64   Cost: -37.104967
Iteration: 65   Cost: -41.495252
Iteration: 66   Cost: -42.131171
Iteration: 67   Cost: -51.938090
Iteration: 68   Cost: -29.028317
Iteration: 69   Cost: -45.032501
Iteration: 70   Cost: -52.998457
Iteration: 71   Cost: -51.752237
Iteration: 72   Cost: -63.582713
Iteration:

Iteration: 91   Cost: -57.688653
Iteration: 92   Cost: -36.887634
Iteration: 93   Cost: -38.030757
Iteration: 94   Cost: -42.081007
Iteration: 95   Cost: -46.823340
Iteration: 96   Cost: -45.576917
Iteration: 97   Cost: -30.793363
Iteration: 98   Cost: -31.978480
Iteration: 99   Cost: -43.712201
Iteration: 0   Cost: -280.839189
Iteration: 1   Cost: -273.768830
Iteration: 2   Cost: -261.536013
Iteration: 3   Cost: -238.489211
Iteration: 4   Cost: -231.064728
Iteration: 5   Cost: -211.741201
Iteration: 6   Cost: -198.453913
Iteration: 7   Cost: -174.630113
Iteration: 8   Cost: -166.631722
Iteration: 9   Cost: -149.157451
Iteration: 10   Cost: -138.015345
Iteration: 11   Cost: -105.730347
Iteration: 12   Cost: -116.532044
Iteration: 13   Cost: -109.270516
Iteration: 14   Cost: -95.198573
Iteration: 15   Cost: -100.789929
Iteration: 16   Cost: -99.127067
Iteration: 17   Cost: -102.205600
Iteration: 18   Cost: -94.333995
Iteration: 19   Cost: -84.445176
Iteration: 20   Cost: -80.233149
Iter

...

Iteration: 8   Cost: -168.310728
Iteration: 9   Cost: -156.595377
Iteration: 10   Cost: -133.706468
Iteration: 11   Cost: -120.334697
Iteration: 12   Cost: -117.648149
Iteration: 13   Cost: -108.146887
Iteration: 14   Cost: -100.940131
Iteration: 15   Cost: -79.494367
Iteration: 16   Cost: -89.662176
Iteration: 17   Cost: -85.245078
Iteration: 18   Cost: -92.634735
Iteration: 19   Cost: -76.179178
Iteration: 20   Cost: -80.825926
Iteration: 21   Cost: -81.044951
Iteration: 22   Cost: -84.227131
Iteration: 23   Cost: -70.993448
Iteration: 24   Cost: -65.088174
Iteration: 25   Cost: -60.246882
Iteration: 26   Cost: -63.544826
Iteration: 27   Cost: -64.673896
Iteration: 28   Cost: -66.730286
Iteration: 29   Cost: -57.731271
Iteration: 30   Cost: -61.362151
Iteration: 31   Cost: -52.232640
Iteration: 32   Cost: -55.620803
Iteration: 33   Cost: -61.523914
Iteration: 34   Cost: -56.427278
Iteration: 35   Cost: -62.626826
Iteration: 36   Cost: -48.479785
Iteration: 37   Cost: -52.390567
Itera

Iteration: 56   Cost: -41.345558
Iteration: 57   Cost: -40.738745
Iteration: 58   Cost: -39.911850
Iteration: 59   Cost: -42.391729
Iteration: 60   Cost: -51.189767
Iteration: 61   Cost: -38.161521
Iteration: 62   Cost: -35.514224
Iteration: 63   Cost: -36.391284
Iteration: 64   Cost: -29.862662
Iteration: 65   Cost: -42.240282
Iteration: 66   Cost: -31.757868
Iteration: 67   Cost: -44.716980
Iteration: 68   Cost: -32.720878
Iteration: 69   Cost: -44.139017
Iteration: 70   Cost: -36.454675
Iteration: 71   Cost: -61.714504
Iteration: 72   Cost: -28.516782
Iteration: 73   Cost: -32.783580
Iteration: 74   Cost: -38.019287
Iteration: 75   Cost: -48.120122
Iteration: 76   Cost: -44.499162
Iteration: 77   Cost: -46.502770
Iteration: 78   Cost: -53.726845
Iteration: 79   Cost: -44.945261
Iteration: 80   Cost: -44.128673
Iteration: 81   Cost: -29.105213
Iteration: 82   Cost: -61.419736
Iteration: 83   Cost: -28.675599
Iteration: 84   Cost: -55.665828
Iteration: 85   Cost: -29.667394
Iteration:

Iteration: 4   Cost: -153.372620
Iteration: 5   Cost: -140.731942
Iteration: 6   Cost: -120.457838
Iteration: 7   Cost: -109.627634
Iteration: 8   Cost: -99.047819
Iteration: 9   Cost: -93.193384
Iteration: 10   Cost: -85.663905
Iteration: 11   Cost: -78.708404
Iteration: 12   Cost: -75.069274
Iteration: 13   Cost: -66.109002
Iteration: 14   Cost: -57.789189
Iteration: 15   Cost: -54.888807
Iteration: 16   Cost: -66.029887
Iteration: 17   Cost: -48.003097
Iteration: 18   Cost: -52.611288
Iteration: 19   Cost: -50.551739
Iteration: 20   Cost: -49.553627
Iteration: 21   Cost: -44.429432
Iteration: 22   Cost: -41.849219
Iteration: 23   Cost: -39.229358
Iteration: 24   Cost: -39.109804
Iteration: 25   Cost: -44.396315
Iteration: 26   Cost: -44.690149
Iteration: 27   Cost: -41.581737
Iteration: 28   Cost: -36.704636
Iteration: 29   Cost: -43.276287
Iteration: 30   Cost: -40.817261
Iteration: 31   Cost: -29.868186
Iteration: 32   Cost: -34.810824
Iteration: 33   Cost: -29.030649
Iteration: 3

Iteration: 51   Cost: -60.946661
Iteration: 52   Cost: -46.616734
Iteration: 53   Cost: -31.859738
Iteration: 54   Cost: -40.982243
Iteration: 55   Cost: -50.065535
Iteration: 56   Cost: -46.438692
Iteration: 57   Cost: -63.194732
Iteration: 58   Cost: -28.800454
Iteration: 59   Cost: -41.021198
Iteration: 60   Cost: -50.638026
Iteration: 61   Cost: -33.151810
Iteration: 62   Cost: -36.148716
Iteration: 63   Cost: -44.735036
Iteration: 64   Cost: -40.445150
Iteration: 65   Cost: -43.099092
Iteration: 66   Cost: -39.902877
Iteration: 67   Cost: -42.582448
Iteration: 68   Cost: -31.425743
Iteration: 69   Cost: -39.617497
Iteration: 70   Cost: -47.463889
Iteration: 71   Cost: -41.835454
Iteration: 72   Cost: -53.457342
Iteration: 73   Cost: -43.424279
Iteration: 74   Cost: -30.894407
Iteration: 75   Cost: -23.000000
Iteration: 76   Cost: -34.296440
Iteration: 77   Cost: -43.432634
Iteration: 78   Cost: -35.094009
Iteration: 79   Cost: -39.448284
Iteration: 80   Cost: -35.339442
Iteration:

Iteration: 0   Cost: -239.691197
Iteration: 1   Cost: -189.190093
Iteration: 2   Cost: -165.677659
Iteration: 3   Cost: -146.386529
Iteration: 4   Cost: -113.611910
Iteration: 5   Cost: -112.967095
Iteration: 6   Cost: -101.240267
Iteration: 7   Cost: -89.068763
Iteration: 8   Cost: -83.394931
Iteration: 9   Cost: -86.495089
Iteration: 10   Cost: -77.460877
Iteration: 11   Cost: -79.201066
Iteration: 12   Cost: -76.161981
Iteration: 13   Cost: -74.383880
Iteration: 14   Cost: -73.556203
Iteration: 15   Cost: -75.675910
Iteration: 16   Cost: -77.283737
Iteration: 17   Cost: -65.276141
Iteration: 18   Cost: -53.838818
Iteration: 19   Cost: -60.408137
Iteration: 20   Cost: -54.800814
Iteration: 21   Cost: -59.483732
Iteration: 22   Cost: -50.881173
Iteration: 23   Cost: -56.963659
Iteration: 24   Cost: -68.867019
Iteration: 25   Cost: -44.796951
Iteration: 26   Cost: -57.373644
Iteration: 27   Cost: -53.114109
Iteration: 28   Cost: -51.885290
Iteration: 29   Cost: -57.347285
Iteration: 30